In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps
from SimPEG import Utils
from SimPEG.Utils import mkvc
import SimPEG.PF as PF
import scipy as sp
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data and topo and build default model
workDir = ''
fName = 'DataGrid.tiff'

dataGrid = DataIO.loadGeoTiffFile(workDir+fName, plotIt=True)

# 2.1.4.	Depth to Source#

There is yet more useful information that we can derive from a suite of high quality magnetic data, again using some simple techniques. It is possible to obtain information about the depth to the top of a magnetic body from magnetic data. 

Analyzing the depth to a magnetic source is valuable for many reasons. Having an approximation of the depth of a body, especially if it is expected to represent ore, is invariably useful for planning drill targets. In the case of some geologic bodies that might be related to mineralization such as porphyry intrusive bodies, the shape or topography of the intrusion can indicate were the regions of highest fluid flow might have occurred. Results of depth to source modelling can be used as a guide and starting point for 3D geological modelling or further exploration of the shape of the anomaly using forward modelling or inversion techniques. Depending on of course the physical properties, specifically the magnetic susceptibility, of the rock types within a study area, and their distribution and orientation, it is possible to find the depths to not only compact magnetic bodies, but possibly depths to the basement in general, or depths to particular stratigraphic horizons.

For this exercise, we will find the depth of both the small high magnetic susceptibility body and the larger magnetic block in the 3D synthetic geologic model. We will use the tilt-depth method (Salem et al., 2007, Blakely et al., 2016) to do this. The tilt angle, discussed in Notebook 2.1.2., considers the vertical and horizontal magnetic gradients. Depth can be estimated where the tilt angle is zero, at the edge or margin of a feature or source. Depth to the edge of a source is then approximately half the distance between the 45deg and -45 degree contours of the calculated tilt angle.

Inclination must be 90, thus reduced to pole data is used in this calculation. This method assumes that the magnetic body is simple and also vertically-sided, which means that the depth estimation will be less accurate for dipping bodies or bodies with complex margins. 



<!-- <img src="./images/SearchQuestII.png"> -->


## Total magnetic intensity data

The below image shows the total magnetic intensity (TMI) that was calculated from the synthetic geologic model. The response is based on magnetic field parameters at the north pole, in other words, the magnetic field is vertical.

In [2]:
selection = ProblemSetter.setDataExtentWidget(dataGrid)
display(selection)

interactive(children=(FloatSlider(value=1304.9632774722995, continuous_update=False, description='East', max=1304.9632774722995, min=-1245.0367225277005, step=500.0), FloatSlider(value=1371.4226190475747, continuous_update=False, description='North', max=1371.4226190475747, min=-1428.5773809524253, step=10.0), FloatSlider(value=40000.0, continuous_update=False, description='Width', max=100000.0, min=1000.0, step=1000.0), FloatSlider(value=40000.0, continuous_update=False, description='Height', max=100000.0, min=1000.0, step=1000.0), Output()), _dom_classes=('widget-interact',))

## Tilt angle

The tilt angle map (also calculated in Notebook 2.1.4) is shown below. The zero contour is expected to represent edges of geological bodies, and is where tilt-depth calculations are focused. 

In [5]:
polylines, attributes = MathUtils.estimateDepth(selection.result)

# Export to shapefile
DataIO.exportShapefile(polylines, attributes, EPSGCode=26909, fileName='Tilt2Depth', label='AverageDepth')

## Tilt-depth

Here we have extracted the zero contour and painted onto the contour the estimated depth to source at that location. The depth is the half-distance calculated between the +/-45 degree contours. The contour can be superimposed on either the TMI data map or the tilt map for comparison. 

The depth of the edges of the smaller magnetic body is estimated at 86 m depth and the depth of the large magnetic block is estimated at 104 m, both estimates are relatively close to the true depths of 100 m for these blocks. The contacts of the smaller and larger magnetic blocks are vertical thus the tilt-depth estimation is expected to be more accurate. The northwest trending fault within the large magnetic block has edges modelled at 73 m depth. This fault dips at 80 degrees and depth to the top of the edge of this feature is possibly less well-estimated due to it's dip. 


In [4]:
scatterData = {}
scatterData['x'] = np.vstack(polylines)[:,0]
scatterData['y'] = np.vstack(polylines)[:,1]
scatterData['z'] = 1
scatterData['c'] = np.concatenate(attributes)
scatterData['cmap'] = 'jet_r'
scatterData['clim'] = [0, 250]

view = Simulator.dataHillsideWidget(selection.result,figName='Tilt2Depth', scatterData=scatterData)
display(view)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=15.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.3, continuous_update=False, description='ColorTransp', max=1.0), FloatSlider(value=1.0, continuous_update=False, description='HSTransp', max=1.0), FloatSlider(value=1.0, continuous_update=False, description='vScale', max=4.0, min=1.0, step=1.0), IntSlider(value=50, continuous_update=False, description='Contours', min=10, step=10), Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'hsv', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), FloatRangeSlider(value=(-317.10699462890625, 917.194580078125), continuous_update=False, description='Color Range', max=917.194580078125, min=-317.10699462890625, readout_format='.1f', step=1.0), Dropdown(description='Color Normalization', index=1, options=('Linear', 'HistEqualized'), value='HistEqualized'), ToggleButton(value=False, description='Export geoTiff', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))